In [11]:
import os
from pathlib import Path
import pandas as pd
from obspy.core import read, UTCDateTime, Stream
import obspy
from preprocessing_tools import *

De igual forma, el catálogo servirá para identificar los eventos que serán procesados, ignorando todo lo demás.

In [12]:
directorio_recortado = "Datos_Trim/"
model = "mex" # Diretorio dentro del cual se colocaran los sinteticos para cada directorio de evento
obs = "Filt" # Directorio dentro del cual se colocaran los sismogramas observados tras haber sido filtrados

#evid_Oaxaca = "2016.06.27.20.50.35" # Sismo mas fuerte
#evid_Guerrero = "2016.07.19.13.42.46"

#evid = evid_Oaxaca
#evdir = directorio_recortado + evid

catalog_sub = 'catalog.dat'
catalogSSN_Oaxaca = 'catalogSSN_Oaxaca.dat'
catalogSSN_Guerrero = 'catalogSSN_Guerrero.dat'

# Generados durante el preprocesamiento
station_file = "station.csv"
ev_st = "ev_st.csv"

#sacdir = "%s/RT"%evdir
#outdir = evdir

#P = Path(sacdir)
df_catalog = read_catalog(catalogSSN_Oaxaca)
stations_meta = pd.read_csv(station_file)
ev_st_df = pd.read_csv(ev_st)

#st = obspy.read(sacdir+'/*')

In [13]:
#ev_row = df_catalog.loc[df_catalog['date_time_id'] == evid_Oaxaca]

In [14]:
#ev_row

In [15]:
#ev_row.iloc[0,0]

In [16]:
# Filter
freqmin = 0.02
freqmax = 0.05
corners = 3

# Desired sampling interval
dt = 1.0

# Reduction velocity in km/sec, 0 sets the reference time to origin time
vred = 0

# time before and after reference time, data will be cut before and after the reference time
time_before = 30
time_after = 200

In [17]:
df_catalog.head(10)

,date_time,lat,lon,depth,mag,corr,date_time_id
0,2016-06-27T20:50:35.000000Z,16.21,-98.00,21.30,5.20,"[0.9316, 51.61, 20160627205035.0]",2016.06.27.20.50.35


In [18]:
for i, row in df_catalog.iterrows():
    evid = row.date_time_id
    evdir = directorio_recortado + evid
    sacdir = "%s/RT"%evdir
    filtdir = "%s/Filt"%evdir
    
    outdir = filtdir
    
    st = obspy.read(sacdir+'/*')
    
    if vred:
        p = 1/vred
    else:
        p = 0

    st.filter("bandpass",freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
    st.taper(max_percentage=0.05)
    origin_time = df_catalog.loc[df_catalog['date_time_id'] == evid]['date_time'].values[0]
    print(origin_time)
    
    Path(filtdir).mkdir(parents=True, exist_ok=True)

    # Trim and decimate the data
    for tr in st:
        tr.decimate(factor=int(tr.stats.sampling_rate*dt), strict_length=False, no_filter=True)
        tr.resample(1/dt, strict_length=False, no_filter=True)
        # Recortar aun mas los datos en base a la distancia entre la estacion y el evento
        tr.stats.sac.t1 = origin_time + p*(tr.stats.sac.dist)
        tr.trim(tr.stats.sac.t1-time_before,tr.stats.sac.t1+time_after,pad=True,fill_value=0)
        tr.data = 100*tr.data # m/s to cm/s
        tr.stats.sac.b = -1*(origin_time - tr.stats.starttime)
        tr.stats.sac.o = 0
        # Save final trace using tdmtpy file name format
        sacout = "%s/%s.%s.dat"%(outdir,tr.stats.sac.kstnm,tr.stats.sac.kcmpnm[-1])
        tr.write(sacout,format="SAC")

2016-06-27T20:50:35.000000Z


# Modelo de Velocidades para obtener más exactitud/precisión

### Probar con otros sismos (checar diario de investigación)

In [19]:
for i, row in df_catalog.iterrows():
    evid = row.date_time_id
    evdir = directorio_recortado + evid
    
    #round(df_catalog.loc[df_catalog['date_time_id'] == evid]["depth"].values[0])]
    depths = sorted([10,15,20,25,30,35,40,45,50,55,60,65,70]) # Rango de profundidades para las funciones de Green
    npts = int(128) # numero de puntos
    t0 = int(0)
    ev_st_sub = ev_st_df.loc[ev_st_df['date_time_id'] == evid].sort_values('st', ascending=True).copy()
    #print(ev_st_sub)

    # Directorio de funciones de Green
    green_dir = "%s/%s"%(evdir,model)
    Path(green_dir).mkdir(parents=True,exist_ok=True)

    for depth in depths:
        dfile = ("{dist:.0f} {dt:.2f} {npts:d} {t0:d} {vred:.1f}\n")
        dfile_out = "%s/dfile"%evdir
        with open(dfile_out,"w") as f:
            # Distancia con cada estacion
            for i, row in ev_st_sub.iterrows():
                f.write(dfile.format(dist=row.distance,dt=dt,npts=npts,t0=t0,vred=vred))

        # Generate the synthetics
        os.system("hprep96 -M %s.d -d %s -HS %.4f -HR 0 -EQEX"%(model,dfile_out,depth))
        !hspec96
        !hpulse96 -D -i > file96
        !f96tosac -B file96

        greens = ("ZDD","RDD","ZDS","RDS","TDS","ZSS","RSS","TSS","ZEX","REX")

        for i,row in stations_meta.iterrows():      
            for j,grn in enumerate(greens):
                sacin = "B%03d%02d%s.sac"%(i+1,j+1,grn)
                sacout = "%s/%s.%.4f"%(green_dir,row.kstnm,depth)
                tmp = read(sacin,format="SAC")
                tmp.filter('bandpass',freqmin=freqmin,freqmax=freqmax,corners=corners,zerophase=True)
                tmp.write("%s.%s"%(sacout,grn),format="SAC") # overwrite

    !\rm B*.sac

  0.1953125E-01  0.1000000E+01
        128         1        65
 ieqex=            0  EARTHQUAKE + EXPLOSION
   10
  TOP  OF MODEL IS FREE SURFACE  
  BASE OF MODEL IS HALFSPACE WITH PROPERTIES OF BOTTOM LAYER
 mex.d
 LAYER             H      P-VEL     S-VEL   DENSITY  
     1         16.000     6.880     3.970     2.820
     2         17.000     7.860     4.540     3.120
     3         77.000     8.110     4.680     3.350
     4          0.000     8.580     4.950     3.420
 Working model
        d         a         b         rho      1/qa      1/qb       bsh    1/qbsh
     16.000     6.880     3.970     2.820  0.001667  0.003333     3.970     2.820
     17.000     7.860     4.540     3.120  0.001667  0.003333     4.540     3.120
     77.000     8.110     4.680     3.350  0.001667  0.003333     4.680     3.350
                8.580     4.950     3.420  0.001667  0.003333     4.950     3.420
 
XLENG=  0.1362000E+04 XFAC=  0.4000000E+01
WAVENUMBER FILTERING bounded in phase velocites
[cma

La librería mttime toma las direcciones de los sismogramas observados y sintéticos como relativas a la ubicación de mtinv.inv (el archivo de configuración para la inversión), así que solo es necesario colocar Filt y gil7.

In [20]:
for i, row in df_catalog.iterrows():
    evid = row.date_time_id
    evdir = directorio_recortado + evid
    filtdir = "%s/%s"%(evdir,obs)
    green_dir = "%s/%s"%(evdir,model)
    
    headers = dict(datetime=row.date_time,
                   longitude=row.lon,
                   latitude=row.lat,
                   depth=",".join([ "%.4f"%d for d in depths]),
                   path_to_data=obs,
                   path_to_green=model,
                   green="herrmann",
                   components="ZRT", # ZRT y ZNE unicas opciones
                   degree=5,
                   weight="distance",
                   plot=0,
                   correlate=0,
                  )
    
    row_st = ev_st_df.loc[ev_st_df['date_time_id'] == evid].sort_values('st', ascending=True)

    pd.options.display.float_format = "{:,.2f}".format
    frame = {"station": row_st["st"]}
    df_out = pd.DataFrame(frame)

    
    df_out["distance"] = row_st["distance"]
    df_out["azimuth"] = row_st["azimuth"]
    df_out["ts"] = int(30)
    df_out["npts"] = npts
    df_out["dt"] = dt
    df_out["used"] = 1
    
    ordered_stations = row_st.merge(stations_meta, left_on=['st', 'ch'], right_on=['kstnm', 'kcmpnm'])[['st', 'stlo', 'stla']]
    
    df_out[["longitude","latitude"]] = ordered_stations[["stlo","stla"]].values
    
    with open(evdir+"/mtinv.in","w") as f:
        for key, value in headers.items():
            f.write("{0:<15}{1}\n".format(key,value))
        f.write(df_out.to_string(index=False))

In [11]:
#!cat mtinv.in